In [1]:
import os
import tensorflow as tf
import pandas as pd

tf.__version__


'2.13.0'

In [2]:
main_dir = r"F:\linux_ubuntu_codes\codes\nlp_lecture"

In [3]:
train_dir = os.path.join(main_dir + r"\train.csv")
test_dir = os.path.join(main_dir + r"\test.csv")

'https://raw.githubusercontent.com/sedeba19/NLP--Random-Models/main/train.csv'

In [4]:
train_dir

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
# Make a train_df
train_df = pd.read_csv(train_dir)
test_df = pd.read_csv(test_dir)
train_df.head()

,id,keyword,location,text,target
3228,4632,emergency%20services,"Sydney, New South Wales",Goulburn man Henry Van Bilsen missing: Emergen...,1
3706,5271,fear,NaN,The things we fear most in organizations--fluc...,0
6957,9982,tsunami,Land Of The Kings,@tsunami_esh ?? hey Esh,0
2887,4149,drown,NaN,@POTUS you until you drown by water entering t...,0
7464,10680,wounds,"cody, austin follows ?*?",Crawling in my skin\nThese wounds they will no...,1


In [6]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac = 1, random_state = 1)
train_df_shuffled.head()

(7613, 5)

In [7]:
train_df_shuffled.shape

target
0    4342
1    3271
Name: count, dtype: int64

In [8]:
# How many examples of each class?
train_df['target'].value_counts()

(7613, 3263)

In [9]:
# How many total samples?
len(train_df), len(test_df)

Target:1 (real disaster)
Text: 
Japan's Restart of Nuclear Reactor Fleet Fast Approaches http://t.co/DbAUjp29Ub
-----

Target:0 (not a real disaster)
Text: 
The media needs to stop publicizing mass murder. So many sick people want the eyes of the world and the media... http://t.co/QZlPFHpwDO
-----

Target:0 (not a real disaster)
Text: 
#hot  Reddit's new content policy goes into effect many horrible subreddits banned or quarantined http://t.co/algtcN8baf #prebreak #best
-----

Target:0 (not a real disaster)
Text: 
@SZMNextDoor I got this cute lil cliff you can fall off of??
-----

Target:0 (not a real disaster)
Text: 
FedEx no longer will transport bioterror germs http://t.co/qfjjDxes7G via @USATODAY
-----


In [ ]:
# Visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text", "target"]][random_index: random_index +5].itertuples():
    _, text, target = row
    if target > 0:
        print(f"Target:{target}", "(real disaster)")
    else:
        print(f"Target:{target}", "(not a real disaster)")
    
    print(f"Text: \n{text}")
    print("-----\n")

### Split data into training and validation sets

In [11]:
from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df['text'],
                                                                            train_df['target'],
                                                                            test_size = 0.1,
                                                                            random_state = 42)

(7613,)

In [12]:
train_df['text'].shape

0       Our Deeds are the Reason of this #earthquake M...
1                  Forest fire near La Ronge Sask. Canada
2       All residents asked to 'shelter in place' are ...
3       13,000 people receive #wildfires evacuation or...
4       Just got sent this photo from Ruby #Alaska as ...
                              ...                        
7608    Two giant cranes holding a bridge collapse int...
7609    @aria_ahrary @TheTawniest The out of control w...
7610    M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...
7611    Police investigating after an e-bike collided ...
7612    The Latest: More Homes Razed by Northern Calif...
Name: text, Length: 7613, dtype: object

In [13]:
train_df["text"]

array(['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
       'Forest fire near La Ronge Sask. Canada',
       "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
       ...,
       'M1.94 [01:04 UTC]?5km S of Volcano Hawaii. http://t.co/zDtoyd8EbJ',
       'Police investigating after an e-bike collided with a car in Little Portugal. E-bike rider suffered serious non-life threatening injuries.',
       'The Latest: More Homes Razed by Northern California Wildfire - ABC News http://t.co/YmY4rSkQ3d'],
      dtype=object)

In [14]:
train_df['text'].to_numpy()

array(['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
       'Forest fire near La Ronge Sask. Canada',
       "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
       ...,
       'M1.94 [01:04 UTC]?5km S of Volcano Hawaii. http://t.co/zDtoyd8EbJ',
       'Police investigating after an e-bike collided with a car in Little Portugal. E-bike rider suffered serious non-life threatening injuries.',
       'The Latest: More Homes Razed by Northern California Wildfire - ABC News http://t.co/YmY4rSkQ3d'],
      dtype=object)

In [15]:
train_df['text'].to_numpy()

(6851, 762, 6851, 762)

In [ ]:
len(train_sentences), len(val_sentences), len(train_labels), len(val_labels)

### Converting text into numbers: Text Vectorization also known as Tokenization

In [17]:
# Get the total words
total_words = 0
for i in train_sentences:
    total_words += len(i.split())

total_words

(6851,)

In [18]:
train_sentences.shape

15

In [19]:
# Get the average words per sentence or line
avg_words_per_sentence = round(total_words/len(train_sentences))
avg_words_per_sentence

In [20]:
# Normally, total number of words is equal or less than the max vocabulary length
max_vocab_length = 10000

# max length of a statement is equal or more than the average word per sentence or line
max_length = 15

In [21]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                    output_mode = "int",
                                    output_sequence_length = max_length)

# Fit the text vectorizer to the training set
text_vectorizer.adapt(train_sentences)

There's a flood in my street! Sentence length is 29.


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[282,   3, 206,   4,  13, 674,   0,   0,   0,   0,   0,   0,   0,
          0,   0]], dtype=int64)>

In [22]:
# Create a sample sentence and tokenize it
samp_sentence = "There's a flood in my street!"
print(samp_sentence, f"Sentence length is {len(samp_sentence)}.")
text_vectorizer([samp_sentence])
#len(tf.squeeze(text_vectorizer([samp_sentence])))

Original text: 

 Has An Ancient Nuclear Reactor Been Discovered In Africa? ÛÒ Your... http://t.co/qadUfO8zXg 

Vectorized text:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  40,   43, 1089,  106,  620,   62, 2512,    4, 1446,  449,   35,
           1,    0,    0,    0]], dtype=int64)>

In [23]:
# Choose random sentence from the training dataset and tokenize it
rand_train_sentence = random.choice(train_sentences)
print(f"Original text: \n\n {rand_train_sentence} \n\nVectorized text:")
text_vectorizer([rand_train_sentence])

['',
 '[UNK]',
 'the',
 'a',
 'in',
 'to',
 'of',
 'and',
 'i',
 'is',
 'for',
 'on',
 'you',
 'my',
 'with',
 'it',
 'that',
 'at',
 'by',
 'this',
 'from',
 'are',
 'be',
 'was',
 'have',
 'like',
 'as',
 'me',
 'but',
 'up',
 'just',
 'so',
 'im',
 'not',
 'amp',
 'your',
 'out',
 'all',
 'after',
 'its',
 'has',
 'no',
 'will',
 'an',
 'fire',
 'when',
 'if',
 'we',
 'get',
 'now',
 'more',
 'via',
 'new',
 'about',
 'what',
 'people',
 'or',
 'news',
 'he',
 'they',
 'over',
 'one',
 'been',
 'how',
 'dont',
 'who',
 'video',
 'into',
 'were',
 'do',
 'us',
 'can',
 'emergency',
 '2',
 'disaster',
 'there',
 'her',
 'some',
 'than',
 'still',
 'would',
 'his',
 'police',
 'crash',
 'burning',
 'suicide',
 'body',
 'california',
 'back',
 'time',
 'buildings',
 'had',
 'why',
 'off',
 'them',
 'got',
 'man',
 'see',
 'storm',
 'know',
 'going',
 'cant',
 'world',
 'first',
 'day',
 'rt',
 'nuclear',
 'love',
 'youtube',
 'our',
 'attack',
 'go',
 'fires',
 'two',
 'their',
 'bomb',

In [24]:
# Get unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
words_in_vocab

10000

In [25]:
len(words_in_vocab)

['', '[UNK]', 'the', 'a', 'in']

In [26]:
# Top 5 words in words_in_vocab
words_in_vocab[:5]

['pakthey', 'pakistan\x89Ûªs', 'pakistans', 'pajamas', 'paints']

In [ ]:
# Least 5 words in words_in_vocab
words_in_vocab[-5:]

### Embedding layer

To make our embedding, we are going to use TensorFlow’s embedding layer: https://www.tensorflow.org/api/docs/python/tf/keras/layers/

The parameters are below mostly used.
* input_dim = size of the vocabulary
* output_dim = size of the output embedding vector.  For example, a value of 100 would mean each token gets represented by a vector 100 long
* input_length = length of the sequences being passed to the embedding layer

In [28]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim = max_vocab_length,
                             output_dim = 64,
                             embeddings_initializer = "uniform",
                             input_length = max_length)

Original text: I was over here dreaming peacefully then that loud ass thunder wanted to scare me. ??

Vectorized text: [[   8   23   60  127 5720 4914  139   16  287  222  281  872    5 4657
    27]]

Vectorized text shape: (1, 15)
Embbedded text: [[[ 4.38363887e-02  1.61645450e-02 -1.19280107e-02  4.27300818e-02
   -1.52721033e-02  2.54082195e-02 -3.46772745e-03  2.12511085e-02
   -4.70531844e-02 -3.58027704e-02  1.06681362e-02  1.07223988e-02
    4.57119085e-02 -6.90912083e-03  1.81941874e-02 -1.14613399e-02
    4.30087000e-03 -6.01671636e-04 -3.05927284e-02 -4.89387289e-02
    3.21773030e-02  2.90276743e-02 -2.46511586e-02 -1.50955841e-03
   -3.41611058e-02  1.74276941e-02  4.25140969e-02 -4.25025821e-02
    3.83951515e-03  2.31327750e-02 -3.21515091e-02 -7.89570808e-03
   -2.11481340e-02 -3.37612852e-02  4.14706506e-02  2.45304815e-02
   -1.58945434e-02 -1.63941383e-02  3.60054262e-02  3.04556154e-02
    3.68481912e-02 -3.67653966e-02 -4.70205806e-02  3.00331004e-02
    2.73678191e

In [ ]:
# Get a random sentence from the training set and return Original text, Vectorized text, Embedded text
rand_sentence = random.choice(train_sentences)
print(f"Original text: {rand_sentence}\n")
print(f"Vectorized text: {text_vectorizer([rand_sentence])}\n")
print(f"Vectorized text shape: {text_vectorizer([rand_sentence]).shape}")
print(f"Embbedded text: {embedding(text_vectorizer([rand_sentence]))}")
print(f"Embedded text shape: {embedding(text_vectorizer([rand_sentence])).shape}")

### Model_3: GRU Model

Another popular and effective RNN component is the Gated Recurrent Unit (GRU) Model. The GRU cell has an LSTM cell but has less parameters.

In [30]:
# Build a RNN Model using GRU Cell

from tensorflow.keras import layers
inputs = layers.Input(shape = (1,), dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64, return_sequences = True)(x)
x = layers.LSTM(32, return_sequences = True)(x)
x = layers.Dense(64, activation = 'relu')(x)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model_3 = tf.keras.Model(inputs, outputs, name = 'model_3_gru')

# Compile the model
model_3.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

Model: "model_3_gru"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVe  (None, 15)                0         
 ctorization)                                                    
                                                                 
 embedding (Embedding)       (None, 15, 64)            640000    
                                                                 
 gru (GRU)                   (None, 15, 64)            24960     
                                                                 
 lstm (LSTM)                 (None, 15, 32)            12416     
                                                                 
 dense (Dense)               (None, 15, 64)            2112      
                                                       

In [31]:
model_3.summary()

Epoch 1/5
215/215 [==============================] - 5s 11ms/step - loss: 0.5180 - accuracy: 0.7383 - val_loss: 0.4718 - val_accuracy: 0.7808
Epoch 2/5
215/215 [==============================] - 2s 9ms/step - loss: 0.3221 - accuracy: 0.8682 - val_loss: 0.5118 - val_accuracy: 0.7782
Epoch 3/5
215/215 [==============================] - 2s 9ms/step - loss: 0.2314 - accuracy: 0.9143 - val_loss: 0.5730 - val_accuracy: 0.7730
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.1708 - accuracy: 0.9413 - val_loss: 0.6247 - val_accuracy: 0.7598
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.1348 - accuracy: 0.9543 - val_loss: 0.7491 - val_accuracy: 0.7585


In [ ]:
# Fit the model
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs = 5,
                              validation_data = (val_sentences, val_labels))



In [ ]:
# Make predictions 
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:5]

In [ ]:
# Convert model_3 pred_prob to labels
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:5]

In [ ]:
# Evaluate the model by comparing the model_3_preds to val_labels
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

model_3_accuracy_score = accuracy_score(val_labels, model_3_preds)
model_3_precision, model_3_recall, model_3_fscore, _ =precision_recall_fscore_support(val_labels,
                                                                                      model_3_preds, average="weighted")

In [ ]:
dict_result = {"model_3_accuracy_score": model_3_accuracy_score,
               "model_3_precision": model_3_precision,
               "model_3_recall": {model_3_recall},
               "model_3_f1score": {model_3_fscore}}
dict_result